In [1]:
!pip install xgboost
import pandas as pd 
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from xgboost.sklearn import XGBClassifier

In [2]:
file1=r"C:\\Users\\CH40068799\\OneDrive - Wipro\\Documents\\Data Science\\ML projects files\\Project 8\\hr_train.csv"

ci=pd.read_csv(file1)
ci.head()


,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.42,0.46,2,150,3,0,1,0,sales,medium
1,0.66,0.77,2,171,2,0,0,0,technical,medium
2,0.55,0.49,5,240,3,0,0,0,technical,high
3,0.22,0.88,4,213,3,1,0,0,technical,medium
4,0.20,0.72,6,224,4,0,1,0,technical,medium


In [3]:
ci.left.value_counts()

0    7424
1    3075
Name: left, dtype: int64

 left: 0-didn't leave, 1-left

In [4]:
cat_cols=ci.select_dtypes(['object']).columns
cat_cols=['sales','salary']
cat_cols

['sales', 'salary']

In [5]:
for col in cat_cols:
    freqs=ci[col].value_counts()
    selected_cats=freqs.index[freqs>500][:-1]
    
    print(col)
    for cat in selected_cats:
        name=col+'_'+cat
        
        ci[name]=(ci[col]==cat).astype(int)
    del ci[col]

sales
salary


In [6]:
ci.shape

(10499, 18)

In [7]:
x_train=ci.drop('left', axis=1)
y_train=ci['left']

In [8]:
x_train.columns

Index(['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident',
       'promotion_last_5years', 'sales_sales', 'sales_technical',
       'sales_support', 'sales_IT', 'sales_product_mng', 'sales_marketing',
       'sales_hr', 'sales_RandD', 'salary_low', 'salary_medium'],
      dtype='object')

In [9]:
del ci

In [10]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [11]:
xgb_params = {  
                "learning_rate":[0.01,0.05,0.1,0.3,0.5],
                "gamma":[i/10.0 for i in range(0,5)],
                "max_depth": [2,3,4,5,6,7,8],
                "min_child_weight":[1,2,5,10],
                "max_delta_step":[0,1,2,5,10],
                "subsample":[i/10.0 for i in range(5,10)],
                "colsample_bytree":[i/10.0 for i in range(5,10)],
                "colsample_bylevel":[i/10.0 for i in range(5,10)],
                "reg_lambda":[1e-5, 1e-2, 0.1, 1, 100], 
                "reg_alpha":[1e-5, 1e-2, 0.1, 1, 100],
                "scale_pos_weight":[1,2,3,4,5,6,7,8,9],
                "n_estimators":[100,500,700,1000]
             }

In [12]:
xgb=XGBClassifier(objective='binary:logistic')

In [14]:
n_iter=10
random_search=RandomizedSearchCV(xgb,n_jobs=-1,cv=5,n_iter=n_iter,scoring='roc_auc',
                                 param_distributions=xgb_params)

In [15]:
random_search.fit(x_train,y_train)

C:\Users\CH40068799\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:21:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100,...
                                        'gamma': [0.0, 0.1, 0.2, 0.3, 0.4],
                                        'learning_rate': [0.01, 0.05, 0.1, 0.3,
                         

In [16]:
report(random_search.cv_results_, 5)

Model with rank: 1
Mean validation score: 0.836 (std: 0.01781)
Parameters: {'subsample': 0.5, 'scale_pos_weight': 8, 'reg_lambda': 0.01, 'reg_alpha': 100, 'n_estimators': 500, 'min_child_weight': 5, 'max_depth': 8, 'max_delta_step': 0, 'learning_rate': 0.3, 'gamma': 0.2, 'colsample_bytree': 0.6, 'colsample_bylevel': 0.9}

Model with rank: 2
Mean validation score: 0.836 (std: 0.01900)
Parameters: {'subsample': 0.8, 'scale_pos_weight': 4, 'reg_lambda': 100, 'reg_alpha': 100, 'n_estimators': 700, 'min_child_weight': 2, 'max_depth': 3, 'max_delta_step': 5, 'learning_rate': 0.3, 'gamma': 0.4, 'colsample_bytree': 0.8, 'colsample_bylevel': 0.6}

Model with rank: 3
Mean validation score: 0.836 (std: 0.01864)
Parameters: {'subsample': 0.7, 'scale_pos_weight': 3, 'reg_lambda': 0.01, 'reg_alpha': 0.01, 'n_estimators': 100, 'min_child_weight': 1, 'max_depth': 5, 'max_delta_step': 10, 'learning_rate': 0.1, 'gamma': 0.2, 'colsample_bytree': 0.9, 'colsample_bylevel': 0.9}

Model with rank: 4
Mean val

In [17]:
random_search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=0.9,
              colsample_bynode=1, colsample_bytree=0.6, gamma=0.2, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.3, max_delta_step=0, max_depth=8,
              min_child_weight=5, missing=nan, monotone_constraints='()',
              n_estimators=500, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=100, reg_lambda=0.01, scale_pos_weight=8, subsample=0.5,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [18]:
xgb_params={
    "n_estimators": [100,500,700,900,1000,1200,1500]
}

In [26]:
xgb1=XGBClassifier(learning_rate=0.1,subsample=0.8,colsample_bylevel=0.8,colsample_bytree=0.8)

In [28]:
print(xgb1)

XGBClassifier(base_score=None, booster=None, colsample_bylevel=0.8,
              colsample_bynode=None, colsample_bytree=0.8, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=0.1, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=0.8, tree_method=None,
              validate_parameters=None, verbosity=None)


In [29]:
from sklearn.model_selection import GridSearchCV

In [31]:
grid_search=GridSearchCV(xgb1,cv=5,param_grid=xgb_params,scoring='roc_auc',verbose=2,n_jobs=-1)

In [32]:
grid_search.fit(x_train,y_train)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


C:\Users\CH40068799\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:34:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=0.8,
                                     colsample_bynode=None,
                                     colsample_bytree=0.8, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=0.1, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=0.8,
                                     tree_method=None, val

In [33]:
report(grid_search.cv_results_,3)

Model with rank: 1
Mean validation score: 0.840 (std: 0.01601)
Parameters: {'n_estimators': 100}

Model with rank: 2
Mean validation score: 0.837 (std: 0.01735)
Parameters: {'n_estimators': 500}

Model with rank: 3
Mean validation score: 0.835 (std: 0.01730)
Parameters: {'n_estimators': 700}



In [34]:
xgb_params={
    "gamma":[4,8,11,12,15],
    "max_depth":[5,6,7,8,9,10,11]
}

In [35]:
xgb2=XGBClassifier(learning_rate=0.1,n_estimators=100,subsample=0.8,min_child_weight=2,colsample_bylevel=0.8,colsample_bytree=0.8)

In [36]:
random_search=RandomizedSearchCV(xgb2,param_distributions=xgb_params,n_iter=20,cv=5,scoring='roc_auc',n_jobs=-1,verbose=2)

In [37]:
random_search.fit(x_train,y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


C:\Users\CH40068799\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:43:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=0.8,
                                           colsample_bynode=None,
                                           colsample_bytree=0.8, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=0.1,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=2, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100, n_jobs=None,
                                           num_parallel_tree=None,
                                           random_state=None, reg_alpha=None,
                                

In [38]:
y_train.value_counts()

0    7424
1    3075
Name: left, dtype: int64

In [39]:
xgb6=XGBClassifier(learning_rate=0.1,n_estimators=100,min_child_weight=2,gamma=12,max_depth=8,
                   scale_pos_weight=1,max_delta_step=0,
                   colsample_bylevel= 1.0, colsample_bytree= 0.8, subsample= 0.7,
                  reg_lambda=1.8,reg_alpha=0)

In [40]:
xgb6.fit(x_train,y_train)

[13:16:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1.0,
              colsample_bynode=1, colsample_bytree=0.8, gamma=12, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=8,
              min_child_weight=2, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1.8, scale_pos_weight=1, subsample=0.7,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [41]:
from sklearn.model_selection import cross_val_score

cross_val_score(xgb6,x_train,y_train,scoring='roc_auc',verbose=10,n_jobs=-1,cv=10)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:   10.4s remaining:   24.3s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   10.7s remaining:   10.7s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:   10.7s remaining:    4.5s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   12.3s finished


array([0.83804083, 0.8417536 , 0.80331764, 0.80971051, 0.85456558,
       0.88331266, 0.8425807 , 0.82823837, 0.83526376, 0.83482445])

In [42]:
scores=[0.83939277, 0.84522644, 0.80372   , 0.81151571, 0.8525834 ,
       0.88518135, 0.84040185, 0.82995162, 0.82688745, 0.83156492]



In [43]:
import numpy as np

In [44]:
np.mean(scores)

0.8366425510000001

In [45]:
np.std(scores)

0.021416045974485785